In [11]:
from sqlalchemy import create_engine
from application import *
from application.models import Metadata
from  sqlalchemy.sql.expression import func
from sqlalchemy.sql import and_

In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [9]:
# get single_work reviews
single_work = Metadata().query.filter(Metadata.review_type == 'single_focus').order_by(func.rand()).all()
len(single_work)

2059

In [12]:
# get stew of non-review, cluster, brief, multi
not_single_work = Metadata().query.filter(Metadata.review_type.in_(["cluster", "really_multi", "not_review", "brief"])).order_by(func.rand()).all()
len(not_single_work)

9882

In [15]:
# make training, test data and labels
trainset = single_work[0:1000] + not_single_work[0:1000]
testset = single_work[1000:2000] + not_single_work[1000:2000]

(2000, 2000)

In [17]:
train_labels = [0 for i in single_work[0:1000]] + [1 for i in not_single_work[0:1000]]
test_labels = [0 for i in single_work[1000:2000]] + [1 for i in not_single_work[1000:2000]]

In [18]:
# process review text .. one hot encoding
trainset_txt = [i.corrected_transcription for i in trainset]
testset_txt = [i.corrected_transcription for i in testset]

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=25,
                                   max_features=10000,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(trainset_txt + testset_txt)


In [27]:
# run tensorflow classifier
# score accuracy
tfidf = tfidf.toarray()

In [28]:
tfidf_train = tfidf[0:2000]
tfidf_test = tfidf[2000:]

In [41]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(tfidf_train, train_labels)


In [42]:
clf.score(tfidf_test, test_labels)

0.8205

In [56]:

import pandas as pd

df = pd.DataFrame()
df['coef'] = clf.coef_[0]
df['term'] = tfidf_vectorizer.get_feature_names()
df = df.sort_values(by="coef")


,coef,term
5584,-2.169950,pp
16,-1.768857,12mo
88,-0.994396,8vo
7411,-0.914740,tile
1291,-0.883461,cloth
78,-0.833014,50
3405,-0.739548,history
4093,-0.712544,labor
4393,-0.711668,lord
6439,-0.696760,says


In [58]:
unknown = Metadata().query.filter(Metadata.review_type == 'needs_audit').order_by(func.rand()).all()


In [62]:
# get text
unknown_txt = [i.corrected_transcription for i in unknown]
# tf_idf

tfidf = tfidf_vectorizer.fit_transform(trainset_txt + testset_txt)
unknown_tfidf = tfidf_vectorizer.transform(unknown_txt)
# generate labels, confidence
result = clf.predict(unknown_tfidf)
#0 is single_work
result

array([1, 0, 0, ..., 1, 1, 1])

In [69]:
resultp = clf.predict_proba(unknown_tfidf)
#resultp[0]
df_unknown = pd.DataFrame(resultp, columns=['single', 'not_single'])
df_unknown['nyt_id'] = [i.nyt_id for i in unknown]
df_unknown = df_unknown.sort_values(by='not_single')
df_unknown.to_csv("probably_single.csv")

In [34]:
model = keras.Sequential()
model.add(keras.layers.Embedding(8500, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          136000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 136,289
Trainable params: 136,289
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(optimizer='Adadelta',
              loss='mean_squared_error',
              metrics=['acc'])

In [40]:
history = model.fit(tfidf_train,
                    train_labels,
                    epochs=20,
                    batch_size=512,
                    validation_data=(tfidf_test, test_labels),
                    verbose=1)

/Users/lavin/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2000 samples, validate on 2000 samples
Epoch 1/20
2000/2000 [==============================] - 2s 984us/step - loss: 0.2501 - acc: 0.5000 - val_loss: 0.2500 - val_acc: 0.5000
Epoch 2/20
2000/2000 [==============================] - 2s 871us/step - loss: 0.2501 - acc: 0.4910 - val_loss: 0.2500 - val_acc: 0.5000
Epoch 3/20
2000/2000 [==============================] - 2s 792us/step - loss: 0.2502 - acc: 0.4840 - val_loss: 0.2500 - val_acc: 0.5000
Epoch 4/20
2000/2000 [==============================] - 2s 862us/step - loss: 0.2502 - acc: 0.4890 - val_loss: 0.2500 - val_acc: 0.5000
Epoch 5/20
2000/2000 [==============================] - 2s 882us/step - loss: 0.2501 - acc: 0.4940 - val_loss: 0.2500 - val_acc: 0.5000
Epoch 6/20
2000/2000 [==============================] - 2s 831us/step - loss: 0.2502 - acc: 0.4770 - val_loss: 0.2500 - val_acc: 0.5000
Epoch 7/20
2000/2000 [==============================] - 2s 808us/step - loss: 0.2501 - acc: 0.4870 - val_loss: 0.2500 - val_acc: 0.5000
